In [2]:
import numpy as np
import networkx as nx
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.sampling import BayesianModelSampling
from scipy.stats import beta, dirichlet

## True Causal Model From Environment:

In [ ]:
model = BayesianModel([('Enfermedad', 'Final'), ('Tratamiento', 'Final'), ('Tratamiento', 'Reaccion'),
                       ('Reaccion', 'Final')])

cpd_e = TabularCPD(variable='Enfermedad', variable_card=2, values=[[0.6, 0.4]])
cpd_tr = TabularCPD(variable='Tratamiento', variable_card=2, values=[[0.5, 0.5]])

cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, 
                   values=[[.7,.4], [.3,.6]], #vivir|farmaco vivir|cirugia #morir|farmaco morir|cirugia
                   evidence=['Tratamiento'],
                   evidence_card=[2])

cpd_f_e_tr_r = TabularCPD(variable='Final', variable_card=2, 
                   values=[[.9,0,.6,0,.3,0,.8,0],  [.1,1,.4,1,.7,1,.2,1]], #vivir|vivir,farmaco,a #morir|
                  evidence=['Enfermedad', 'Tratamiento','Reaccion'],
                  evidence_card=[2, 2,2])

model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)

infer = VariableElimination(model)
sampling = BayesianModelSampling(model)

In [ ]:
class true_causal_model:
    def __init__(self,N,infer,sampling):
        self.num_variables=N
        self.infer=infer
        self.sampling=sampling
        
   # def action_simulator(self,chosen_action):
   #     lista=list(sampling.forward_sample(size=1, return_type='sampled'))
   #     response=dict()
   #     lista_2=list(lista[0])
   #     response['Enfermedad']=lista_2[0]
   #     response['Tratamiento']=lista_2[1]
   #     response['Reaccion']=lista_2[2]
   #     response['Final']=lista_2[3]
   #     return(response)
    
    def action_simulator(self,chosen_action):
        response=dict()
        response['Enfermedad']=np.random.randint(0,2)
        response['Tratamiento']=chosen_action
        response['Reaccion']=self.infer.map_query(['Reaccion'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad']})['Reaccion']
        response['Final']=self.infer.map_query(['Final'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad'],'Reaccion':response['Reaccion']})['Final']
        return(response)

### Ejemplo de uso

In [ ]:
modelo_causal=true_causal_model(4,infer,sampling)

In [ ]:
modelo_causal.action_simulator(1)

## Agent and beliefs

In [ ]:
class causal_agent:
    def __init__(self, true_causal_model, max_variables):
        self.max_variables=max_variables
        self.true=true_causal_model
        self.beliefs=dict()
    
    def belief_random_formation(self):
        enf_0=beta.rvs(1, 2, size=1)[0]
        self.beliefs['Enfermedad']=[enf_0, 1-enf_0]
        
        tr_0=beta.rvs(1, 2, size=1)[0]
        self.beliefs['Tratamiento']=[tr_0, 1-tr_0]
        
        r_1=beta.rvs(1, 2, size=1)[0]
        r_2=beta.rvs(1, 2, size=1)[0]
        self.beliefs['Reaccion|Tratamiento']=[[r_1,r_2],
                                              [1-r_1,1-r_2]]
        
        f_1=beta.rvs(1, 2, size=1)[0]
        f_2=beta.rvs(1, 2, size=1)[0]
        f_3=beta.rvs(1, 2, size=1)[0]
        f_4=beta.rvs(1, 2, size=1)[0]
        f_5=beta.rvs(1, 2, size=1)[0]
        f_6=beta.rvs(1, 2, size=1)[0]
        f_7=beta.rvs(1, 2, size=1)[0]
        f_8=beta.rvs(1, 2, size=1)[0]
        self.beliefs['Final']=[[f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8],
                               [1-f_1,1-f_2,1-f_3,1-f_4,1-f_5,1-f_6,1-f_7,1-f_8]]
        return(self.beliefs)
        
    def prob_data_given_beliefs(self,datos, self.beliefs):
        #Esto se debe usar al final de cada juego
        #datos es un diccionario Enfermedad Tratamiento Reaccion Final
        #Entonces P(Enfermedad,Tratamiento,Reaccion,Final)=P(Final|Enfermedad,Tratamiento,Reaccion)P(Tratamiento|Reaccion)P(Tratamiento)P(Enfermedad)
        #datos es una lista [1,0,0,1] en el orden Enfermedad, Tratamiento, Reaccion, Final
        
        #Para Tratamiento:
        prob_enfermedad=self.beliefs['Enfermedad'][datos[0]]
        
        #Para Tratamiento
        prob_tratamiento=self.beliefs['Tratamiento'][datos[1]]
        
        #Para reaccion dado tratamiento
        prob_tr_reac = self.beliefs['Reaccion|Tratamiento'][datos[2]][datos[1]]
        
        #Para final dado enfermedad, tratamiento y reaccion:
        prob_final = self.beliefs['Final'][datos[3]][datos[2]][datos[1]][datos[0]]
        
        probability=prob_enfermedad*prob_tratamiento*prob_tr_reac*prob_final
        
        return(probability)
    
    
    def belief_bayesian_updating(self,datos,prior_beliefs):
        proba_datos=prob_data_given_beliefs(datos,prior_beliefs)
        self.beliefs['Enfermedad'][datos[0]]=proba_datos*prior_beliefs['Enfermedad'][datos[0]]
        self.beliefs['Tratamiento'][datos[1]]=proba_datos*prior_beliefs['Tratamiento'][datos[1]]
        self.beliefs['Reaccion|Tratamiento'][datos[2]][datos[1]]=proba_datos*prior_beliefs['Reaccion|Tratamiento'][datos[2]][datos[1]]
        self.beliefs['Final'][datos[3]][datos[2]][datos[1]][datos[0]]=proba_datos*prior_beliefs[datos[3]][datos[2]][datos[1]][datos[0]]
        
    def fix_causal_model(self):
        model=self.true.copy()
        cpd_e=TabularCPD(variable='Enfermedad', variable_card=2, values=self.beliefs['Enfermedad'])
        cpd_tr=TabularCPD(variable='Tratamiento', variable_card=2, values=self.beliefs['Tratamiento'])
        cpd_r_tr=cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, values=self.beliefs['Reaccion|Tratamiento'], evidence=['Tratamiento'],evidence_card=[2])
        cpd_f_e_tr_r= TabularCPD(variable='Final', variable_card=2, values=self.beliefs['Final'],evidence=['Enfermedad', 'Tratamiento','Reaccion'],evidence_card=[2, 2,2])
        model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)
        inference = VariableElimination(model)
        
        
    def decision_making():

## Use

In [ ]:
modelo=true_causal_model(4,infer,sampling)
agent=causal_agent(modelo,4)

In [ ]:
# Luego con true_causal_model.action_simulator voy a simular una accion
# Con lo que se observe, voy a usar causal_agent.belief_updating, luego fix model, luego make choice, etc
# Todo esto en un loop cuya ultima linea sea sumar el reward obtenido 

In [ ]:
from scipy.stats import beta
beta.rvs(1, 2, size=1)[0]

In [ ]:
r[0]

In [ ]:
[beta.rvs(1, 2, size=1), 1]

In [ ]:
test=[[1,2],[3,4]]

In [ ]:
test[0][1]

In [3]:
np.zeros(5,dtype='float')

array([ 0.,  0.,  0.,  0.,  0.])